In [1]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import csv

access_key = "BQCEBxy7nnWRMmOMKh8czBzunWnJrecSzjTQKaddbv_uxK3hHS-nMc3w7msDF7LUvHWAF2u83BYQFIatzH_3IQHbXC8hVb83LNfLLKd-ZHeuTqGtLS8";# this will expire and need to be updated
client_id = "871b7182f56b41e98864ba7675511134"
client_secret = "9433c21a4ccb42e8a9fafe5a1f86e0da"



cache_path = "cached_token.txt" #this is all to just update my personal token information to use spoify API 

sp_oauth = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://localhost:3000",
    scope="user-top-read",
    cache_path=cache_path
)

try:
    # Attempt to load cached token
    token_info = sp_oauth.get_cached_token()

    # Refresh token if needed
    if token_info and sp_oauth.is_token_expired(token_info):
        token_info = sp_oauth.refresh_access_token(token_info["refresh_token"])

    # Create Spotify object with valid access token
    spotifyToken = spotipy.Spotify(auth=token_info["access_token"])

except spotipy.SpotifyException as e:
    print("Spotify error:", e)
    # Handle Spotify-specific errors (e.g., re-authentication)
except Exception as ex:
    print("Unexpected error:", ex)
    # Handle other unexpected errors


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id= client_id,client_secret=client_secret, redirect_uri="http://localhost:3000", scope="user-library-read"))

topTracks = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id= client_id,client_secret=client_secret, redirect_uri="http://localhost:3000", scope="user-top-read"))

top_tracks = topTracks.current_user_top_tracks(offset=0, limit=50, time_range="medium_term")



limit = 20  #20 is spotifys limit for getting songs
offset = 0
all_tracks = []
song_ids = []
artists = []
all_genres = []
names = []
artist_ids = []

while True:
    results = sp.current_user_saved_tracks(limit=limit, offset=offset)
    
    if not results['items']:
        break  # No more items to retrieve
    
    all_tracks.extend(results['items'])
    song_ids.extend([track['track']['id'] for track in results['items']])
    artists.extend([track['track']['artists'] for track in results['items']])#get track information
    names.extend((track['track']['name']) for track in results['items'])
    offset += limit
    

batch_size = 100
offset = 0
song_info = []

for i in range(0, len(song_ids), batch_size):
    batch_ids = song_ids[i:i + batch_size]
    audio_features = sp.audio_features(batch_ids)#get specific audio features in batches due to api limits

    for j, features in enumerate(audio_features):
        song_id = batch_ids[j]
        row = [song_id, features]
        song_info.append(row)



Unexpected error: 'NoneType' object is not subscriptable


This was my function to attempt to get the artists genres- it was working at one point with smaller data sets but after attempting to process larger data sets I was getting call limits

#def process_artists_in_batches(artists, batch_size):
    for i in range(0, len(artists), batch_size):
        batch = artists[i:i+batch_size]
        for artist in batch:
            artistInfo = artist[0]
            artist_id = artistInfo['id']
            print(artist_id)
            artist_info = sp.artist(artist_id)
            artist_genres = artist_info['genres']
            all_genres.append(artist_genres)

            
        

song_info['artist_genres'] = song_info['artist_genres'].apply(lambda x: ast.literal_eval(x)[0] if (pd.notna(x) and ast.literal_eval(x)) else x)
song_info['artist_genres'] = all_genres.tolist()    


In [2]:
import csv

header = [
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'time_signature'
]
filename = "song_features_with_names.csv"

with open(filename, 'w', newline= '', encoding = 'utf-8') as csvfile:

    writer = csv.writer(csvfile)

    writer.writerow(header)

    for song in song_info:
        audio_features = song[1]
        values = [str(audio_features.get(key, '')) for key in header]
        writer.writerow(values)
            

In [3]:
import pandas as pd


df = pd.read_csv("unlikedSongs.csv")


columns_to_remove = ['artist_id', 'artist_genres', 'artist_name', 'track_id', 'track_popularity', 'artist_popularity', 'playlist_url', 'album', 'duration_ms', 'year']


df = df.drop(columns=columns_to_remove)


header = df.columns.tolist()

df.to_csv("modified_playlist2.csv", index=False, header=header)

print("Columns removed and saved to modified_playlist.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'unlikedSongs.csv'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
file_path = 'song_features_with_names.csv' 

column_names = [
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo', 'time_signature'
]

data = pd.read_csv(file_path, names=column_names)


# Convert columns to numeric
data = data.apply(pd.to_numeric, errors='coerce')
names.insert(0,'NaN')
data.insert(loc=0,column='names',value=names)




In [ ]:
import seaborn as sns
import numpy as np

def replace_with_column_mean(df):
  
    for column in df.columns:
       
        if (df[column] == 0).any() or df[column].isnull().any():
            
            column_mean = df[column][(df[column] != 0) & (~df[column].isnull())].mean()
            
            
            df[column] = df[column].replace({0: column_mean, np.nan: column_mean})
    
    return df

otherData = pd.read_csv('modified_playlist2.csv')

replace_with_column_mean(otherData)# cleaning data 
replace_with_column_mean(data)

df['track_name'] = df['track_name'].str.replace(',', '', regex=True)# take out comas so it can be processed in map reduce properly 

print("Liked songs data")
data.hist(figsize=(12, 10))
plt.tight_layout()
otherData.hist(figsize=(12, 10))
plt.tight_layout()

otherData.to_csv("modified_playlist2.csv", index= False)
data.to_csv("song_features_with_names.csv", index= False)
